# 컨테이너 기본 개념 및 실습
이 노트북에서는 컨테이너의 기본 개념, 컨테이너 이미지 빌드 및 실행, 그리고 Azure Container Registry(ACR)로 이미지를 푸시하는 방법을 실습합니다.

## 1. 컨테이너란 무엇인가?
컨테이너는 애플리케이션과 그 실행 환경을 하나의 패키지로 묶어 어디서나 일관되게 실행할 수 있도록 해주는 기술입니다.

### 컨테이너의 주요 이점
- **이식성**: 어디서나 동일하게 실행 가능
- **경량화**: 가상머신보다 리소스 사용이 적음
- **빠른 배포 및 확장**: 이미지 기반으로 빠르게 배포 및 확장 가능
- **격리성**: 애플리케이션 간 충돌 방지

### VM vs 컨테이너 아키텍처 상세 비교

가상머신(VM)과 컨테이너는 모두 애플리케이션을 격리하여 실행하는 기술이지만, 구조와 동작 방식에 차이가 있습니다.

#### 1. VM 아키텍처

아래는 VM 기반 아키텍처의 구조를 나타낸 다이어그램입니다.

```
[앱1]         [앱2]
   |              |
[게스트 OS1]  [게스트 OS2]
   \           /
   [하이퍼바이저]
       |
   [호스트 OS]
       |
   [하드웨어]
```

- 각 VM은 별도의 게스트 OS를 포함하므로 무겁고, 리소스 사용이 많습니다.
- 하이퍼바이저가 하드웨어와 게스트 OS 사이에서 가상화를 담당합니다.

#### 2. 컨테이너 아키텍처

컨테이너 기반 아키텍처의 구조는 다음과 같습니다.

```
   [앱1]      [앱2]
     |           |
[컨테이너1] [컨테이너2]
   |         |
[컨테이너 엔진]
     |
[호스트 OS]
     |
[하드웨어]
```

- 컨테이너는 호스트 OS의 커널을 공유하며, 별도의 OS가 필요하지 않아 경량화되어 있습니다.
- 컨테이너 엔진(예: Docker)이 컨테이너의 실행과 관리를 담당합니다.

#### 3. 비교 요약

| 항목         | VM                                 | 컨테이너                        |
|--------------|-------------------------------------|---------------------------------|
| OS           | 각 VM마다 별도 게스트 OS 필요        | 호스트 OS 커널 공유             |
| 리소스 사용  | 무거움, 오버헤드 큼                  | 경량, 오버헤드 적음             |
| 시작 속도    | 느림                                | 빠름                            |
| 이식성       | 제한적                               | 매우 높음                       |
| 격리 수준    | 강력한 격리(보안성 높음)             | 프로세스 수준 격리              |

### 컨테이너 아키텍처
컨테이너는 호스트 OS의 커널을 공유하며, 각 컨테이너는 독립적으로 실행됩니다. 대표적인 컨테이너 엔진으로는 Docker가 있습니다.

## 2. 애플리케이션 준비

> **참고:** 이 실습 가이드는 [Spring 공식 Docker 가이드](https://spring.io/guides/gs/spring-boot-docker)를 기반으로 작성되었습니다. 보다 자세한 내용이나 최신 정보를 원한다면 해당 링크를 참고해 주세요.

이 섹션에서는 Spring Boot 애플리케이션을 예시로, 소스코드 작성부터 컨테이너 이미지 빌드, 실행까지의 전체 과정을 단계별로 실습합니다.

### 2.1 Maven 설치 및 환경 준비
Spring Boot 프로젝트를 빌드하고 컨테이너 이미지를 만들기 위해 Maven이 필요합니다. 아래는 macOS에서 Maven을 설치하는 방법입니다.

```bash
brew install maven
```

설치가 완료되면 아래 명령어로 Maven 버전을 확인할 수 있습니다.

```bash
mvn -v
```

- Windows: [Maven 공식 다운로드 페이지](https://maven.apache.org/download.cgi)에서 zip 파일을 받아 압축 해제 후 환경변수 설정
- Linux: 패키지 매니저(예: apt, yum)로 설치 가능. 예시: `sudo apt-get install maven`

---

### 2.2 Spring Boot 프로젝트 생성
아래 명령어로 Maven 기반 Spring Boot 프로젝트를 생성할 수 있습니다.

```bash
mvn archetype:generate \
  -DgroupId=com.example.springbootdocker \
  -DartifactId=springboot-docker-demo \
  -DarchetypeArtifactId=maven-archetype-quickstart \
  -DinteractiveMode=false
```

---

### 2.3 프로젝트 폴더 구조 확인 및 이동
프로젝트가 생성되면 폴더 구조를 확인하고, 작업 디렉터리를 프로젝트 루트로 이동합니다.

```bash
cd springboot-docker-demo
ls -R
```

---

### 2.4 pom.xml 작성 및 의존성 확인
아래는 예시 pom.xml 파일입니다. (기존 pom.xml을 아래와 같이 수정하거나, 의존성을 추가하세요)

```xml
<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
         xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
    <modelVersion>4.0.0</modelVersion>
    <groupId>com.example</groupId>
    <artifactId>demo</artifactId>
    <version>0.0.1-SNAPSHOT</version>
    <packaging>jar</packaging>
    <name>demo</name>
    <description>Demo project for Spring Boot</description>
    <parent>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-parent</artifactId>
        <version>3.2.5</version>
        <relativePath/>
    </parent>
    <properties>
        <java.version>17</java.version>
    </properties>
    <dependencies>
        <dependency>
            <groupId>org.springframework.boot</groupId>
            <artifactId>spring-boot-starter-web</artifactId>
        </dependency>
        <dependency>
            <groupId>org.springframework.boot</groupId>
            <artifactId>spring-boot-starter-test</artifactId>
            <scope>test</scope>
        </dependency>
    </dependencies>
    <build>
        <plugins>
            <plugin>
                <groupId>org.springframework.boot</groupId>
                <artifactId>spring-boot-maven-plugin</artifactId>
            </plugin>
        </plugins>
    </build>
</project>
```

---

### 2.5 애플리케이션 코드 작성 (Application.java)
아래 코드를 src/main/java/com/example/springbootdocker/Application.java 파일로 저장하세요.

```java
package com.example.springbootdocker;

import org.springframework.boot.SpringApplication;
import org.springframework.boot.autoconfigure.SpringBootApplication;
import org.springframework.web.bind.annotation.GetMapping;
import org.springframework.web.bind.annotation.RestController;

@SpringBootApplication
@RestController
public class Application {
    @GetMapping("/")
    public String home() {
        return "Hello Docker World";
    }
    public static void main(String[] args) {
        SpringApplication.run(Application.class, args);
    }
}
```

---

## 3. 컨테이너 이미지 빌드 및 테스트

### 3.1 Dockerfile 작성
아래 코드를 Dockerfile로 저장하세요.

```dockerfile
FROM openjdk:17-jdk-slim
ARG JAR_FILE=target/*.jar
COPY ${JAR_FILE} app.jar
ENTRYPOINT ["java","-jar","/app.jar"]
```

---

### 3.2 애플리케이션 빌드(JAR 생성)
```bash
mvn clean package
```

---

### 3.3 Docker 이미지 빌드
```bash
docker build -t myapp:latest .
```

---

### 3.4 빌드된 이미지 확인 및 테스트
```bash
docker images
docker run -d -p 8080:8080 --name myapp myapp:latest
```
---

## 4. 컨테이너 실행 실습
빌드한 이미지를 기반으로 컨테이너를 실행해봅니다.

In [ ]:
# 컨테이너 실행
# 8080번 포트를 호스트에 바인딩하여 실행 (Spring Boot 기본 포트)
docker run -d -p 8080:8080 --name myapp myapp:latest

## 5. Azure Container Registry(ACR)로 이미지 푸시
Azure에 컨테이너 이미지를 저장하려면 ACR을 사용합니다.

### ACR에 로그인

In [ ]:
# Azure CLI로 로그인
az login

# ACR에 로그인 (예시: myacr 이름의 레지스트리)
az acr login --name myacr

In [ ]:
# 이미지 태깅 (ACR 주소로)
docker tag myapp:latest myacr.azurecr.io/myapp:latest

# 이미지 푸시
docker push myacr.azurecr.io/myapp:latest

## 6. 정리

이 노트북에서는 컨테이너의 기본 개념부터 실제 Spring Boot 애플리케이션을 컨테이너 이미지로 빌드하고, 실행 및 Azure Container Registry(ACR)로 푸시하는 전체 과정을 실습했습니다.

>다음 단계에서는 빌드한 이미지를 AKS(Azure Kubernetes Service) 클러스터에 배포하는 방법을 실습합니다.